In [4]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import seaborn as sns
from matplotlib import axes
import pandas as pd
import os
from os.path import join
import json

In [5]:

import sys, os
sys.path.append('/cluster/home/kamara/Explain')
from clutils.nbutils import *
os.getcwd()

'/cluster/home/kamara/Explain/exp_synthetic'

In [6]:
def parseLogs(logfile, kw="rawlogs:"):
    logs = {}
    dicts = []
    with open(logfile) as f:
        for i_line, line in enumerate(f):
            pos = line.find(kw)
            if pos >= 0:
                pos += len(kw)
                pos_tensor = line.find("tensor(")
                while pos_tensor >= 0:
                    comma_end = line.find(",", pos_tensor)
                    tensor_end = line.find(")", pos_tensor)
                    line_new = line[:pos_tensor] + line[pos_tensor + len("tensor("):comma_end] + line[tensor_end+1:]
                    print("LINE", line)
                    print("NEW", line_new)
                    if len(line_new) >= len(line):
                        print("No line reduction")
                        break
                    line = line_new
                    pos_tensor = line.find("tensor(")
                    
                #print(ast.literal_eval(line[pos:]))
                #line[pos:] = line[pos:].replace("'", "")
                kline = line[pos:]
                kline = kline.replace("true", "True")
                try:
                    dic = ast.literal_eval(kline)
                except:
                    # print("Encountering weird patterns in logs")
                    # print("Line number %d" % i_line)
                    # print(line)
                    line = line.replace("nan,", "-1e8,")
                    line = line.replace("NaN", "-1e8")
                    try:
                        dic = ast.literal_eval(line[pos:])
                    except:
                        print("Unable to replace NaNs")
                        print(line)
                        continue
                for k in dic.keys():
                    if k not in logs:
                        logs[k] = []
                    logs[k].append(dic[k])
                dicts.append(dic)

    df = pd.DataFrame(dicts)

    return logs, df


In [7]:
def get_info(logdir):
    infos, accuracy_top, accuracy, fidelity = {}, {}, {}, {}
    for filename in os.listdir(logdir):
        if filename.endswith(".stdout") and filename.startswith("_"):
            _, infos[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__infos:')
            _, accuracy_top[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__accuracy_top:')
            _, accuracy[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__accuracy:')
            _, fidelity[filename.replace(".stdout", "")] = parseLogs(join(logdir, filename), kw='__fidelity:')             
    return(infos, accuracy_top, accuracy, fidelity)

def get_param_ranges(jsonpath, dataset = ""):
    f = open(jsonpath,)
    data = json.load(f)
    ranges = data['params']
    ranges['none'] = ''
    return(ranges)   

In [18]:
def get_df_results(logs, ranges, name, metrics, selection = 'last'):
    dicts = []
    for params_set in enumerateParams(ranges):
        key = name.format(**params_set)
        # if key in logs and len(logs[key]) >= 1 and metric in logs[key]:
        any_metric = (key in logs) and (len(logs[key]) >= 1) and any([metric in logs[key] for metric in metrics])
        if any_metric:
            if selection == 'all':
                for index, row in logs[key].iterrows():
                    metrics_dict = {
                        metric: select_value(row.to_frame().T, metric, 'last') if key in logs and len(logs[key]) >= 1 and metric in logs[key] else -1
                        for metric in metrics
                    }
                    dicts.append(dictmerge(params_set, metrics_dict))
                    
            else:
                metrics_dict = {
                    metric: select_value(logs[key], metric, selection) if key in logs and len(logs[key]) >= 1 and metric in logs[key] else -1
                    for metric in metrics
                }
                dicts.append(dictmerge(params_set, metrics_dict))

    df = pd.DataFrame(dicts)
    #df = df.drop(columns=['none'])
    return df

## Syn1

In [26]:
expe = "syn1"

logdir = f'/cluster/home/kamara/Explain/checkpoints/{expe}_gpu1/logs'
jsonpath = f'/cluster/home/kamara/Explain/config_{expe}.json'
infos, accuracy_top, accuracy, fidelity = get_info(logdir)

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}'

ranges = get_param_ranges(jsonpath)
df_infos = get_df_results(infos, ranges, name, metrics = ['time', 'edge_mask size'])
df_acc_top = get_df_results(accuracy_top, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'auc'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 'mask_sparsity', 'expl_edges'])

#scores = pd.concat([df_acc, df_fid], axis=1, join="inner")
scores = pd.merge(df_acc_top, df_acc, suffixes=['_top', ''], on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])
scores = pd.merge(scores, df_fid, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])


_explainer_name=random_sparsity=0.997


In [27]:
res = scores.groupby(by=['dataset', 'sparsity', 'explainer_name']).mean()
res

num_test_nodes  gpu  f1_score_top  \
dataset sparsity explainer_name                                      
syn1    0.700    distance                 200.0  1.0      0.684318   
                 gnnexplainer             200.0  1.0      0.324167   
                 ig_node                  200.0  1.0      0.438333   
                 occlusion                200.0  1.0      0.054167   
                 pagerank                 200.0  1.0      0.868333   
                 pgmexplainer             200.0  1.0      0.570303   
                 random                   200.0  1.0      0.003333   
                 sa_node                  200.0  1.0      0.018788   
        0.990    distance                 200.0  1.0      0.698606   
                 gnnexplainer             200.0  1.0      0.339591   
                 ig_node                  200.0  1.0      0.438333   
                 occlusion                200.0  1.0      0.054167   
                 pagerank                 200.0  1.0      0.868333   
                 pgmexplainer             200.0  1.0      0.561667   
                 random                   200.0  1.0      0.001667   
                 sa_node                  200.0  1.0      0.018788   
        0.997    distance                 200.0  1.0      0.698606   
                 gnnexplainer             200.0  1.0      0.324076   
                 ig_node                  200.0  1.0      0.438333   
                 occlusion                200.0  1.0      0.054167   
                 pagerank                 200.0  1.0      0.868333   
                 pgmexplainer             200.0  1.0      0.572500   
                 sa_node                  200.0  1.0      0.018788   

                                 recall_top  precision_top     ged   auc_top  \
dataset sparsity explainer_name                                                
syn1    0.700    distance          0.675000       0.695500   3.955  0.998328   
                 gnnexplainer      0.317500       0.332167   8.615  0.964629   
                 ig_node           0.438333       0.438333   4.915  0.494942   
                 occlusion         0.054167       0.054167   9.720  0.044283   
                 pagerank          0.868333       0.868333   2.355  0.999883   
                 pgmexplainer      0.570000       0.570667   5.060  0.963200   
                 random            0.003333       0.003333  14.500  0.492276   
                 sa_node           0.018333       0.019333   9.240  0.041694   
        0.990    distance          0.686667       0.713333   3.920  0.998173   
                 gnnexplainer      0.331667       0.349250   8.470  0.964168   
                 ig_node           0.438333       0.438333   4.915  0.494942   
                 occlusion         0.054167       0.054167   9.720  0.044283   
                 pagerank          0.868333       0.868333   2.355  0.999883   
                 pgmexplainer      0.561667       0.561667   4.925  0.965568   
                 random            0.001667       0.001667  14.430  0.488264   
                 sa_node           0.018333       0.019333   9.240  0.041694   
        0.997    distance          0.686667       0.713333   3.920  0.998173   
                 gnnexplainer      0.317500       0.332417   8.500  0.964230   
                 ig_node           0.438333       0.438333   4.915  0.494942   
                 occlusion         0.054167       0.054167   9.720  0.044283   
                 pagerank          0.868333       0.868333   2.355  0.999883   
                 pgmexplainer      0.572500       0.572500   4.920  0.963176   
                 sa_node           0.018333       0.019333   9.240  0.041694   

                                 f1_score    recall  precision       auc  \
dataset sparsity explainer_name                                            
syn1    0.700    distance        0.011780  1.000000   0.005925  0.998328   
                 gnnexplainer    0.447869  0.991667  

In [28]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/syn1_gpu1.csv', index=False)

## Syn6

In [32]:
expe = "syn6"

logdir = f'/cluster/home/kamara/Explain/checkpoints/{expe}_gpu1/logs'
jsonpath = f'/cluster/home/kamara/Explain/config_{expe}.json'
infos, accuracy_top, accuracy, fidelity = get_info(logdir)

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}'
print(accuracy_top)
ranges = get_param_ranges(jsonpath)
df_infos = get_df_results(infos, ranges, name, metrics = ['time', 'edge_mask size'])
df_acc_top = get_df_results(accuracy_top, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'auc'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-', 'mask_sparsity', 'expl_edges'])

#scores = pd.concat([df_acc, df_fid], axis=1, join="inner")
scores = pd.merge(df_acc_top, df_acc, suffixes=['_top', ''], on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])
scores = pd.merge(scores, df_fid, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])
scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])


_explainer_name=random_sparsity=0.99
{'_explainer_name=random_sparsity=0.7':    recall  precision  f1_score     ged       auc
0   0.001      0.001     0.001  10.715  0.512361, '_explainer_name=random_sparsity=0.99':    recall  precision  f1_score    ged       auc
0   0.002      0.002     0.002  10.58  0.506105, '_explainer_name=random_sparsity=0.997':    recall  precision  f1_score     ged       auc
0   0.002      0.002     0.002  10.675  0.510506, '_explainer_name=distance_sparsity=0.7':    recall  precision  f1_score    ged      auc
0   0.724     0.7255  0.724667  2.395  0.99905, '_explainer_name=distance_sparsity=0.99':    recall  precision  f1_score    ged      auc
0   0.724     0.7255  0.724667  2.395  0.99905, '_explainer_name=distance_sparsity=0.997':    recall  precision  f1_score    ged      auc
0   0.724     0.7255  0.724667  2.395  0.99905, '_explainer_name=pagerank_sparsity=0.99':    recall  precision  f1_score   ged       auc
0   0.599    0.74425  0.663556  1.49  0.999769,

In [33]:
res = scores.groupby(by=['dataset', 'sparsity', 'explainer_name']).mean()
res

num_test_nodes  gpu  f1_score_top  \
dataset sparsity explainer_name                                      
syn6    0.700    distance                 200.0  1.0      0.724667   
                 gnnexplainer             200.0  1.0      0.211556   
                 ig_node                  200.0  1.0      0.199222   
                 occlusion                200.0  1.0      0.023000   
                 pagerank                 200.0  1.0      0.663556   
                 pgmexplainer             200.0  1.0      0.391778   
                 random                   200.0  1.0      0.001000   
                 sa_node                  200.0  1.0      0.113222   
        0.990    distance                 200.0  1.0      0.724667   
                 gnnexplainer             200.0  1.0      0.248778   
                 ig_node                  200.0  1.0      0.199222   
                 occlusion                200.0  1.0      0.023000   
                 pagerank                 200.0  1.0      0.663556   
                 pgmexplainer             200.0  1.0      0.384556   
                 random                   200.0  1.0      0.002000   
                 sa_node                  200.0  1.0      0.113222   
        0.997    distance                 200.0  1.0      0.724667   
                 gnnexplainer             200.0  1.0      0.199556   
                 ig_node                  200.0  1.0      0.199222   
                 occlusion                200.0  1.0      0.023000   
                 pagerank                 200.0  1.0      0.663556   
                 pgmexplainer             200.0  1.0      0.394222   
                 random                   200.0  1.0      0.002000   
                 sa_node                  200.0  1.0      0.113222   

                                 recall_top  precision_top     ged   auc_top  \
dataset sparsity explainer_name                                                
syn6    0.700    distance             0.724        0.72550   2.395  0.999050   
                 gnnexplainer         0.211        0.21225   5.940  0.974278   
                 ig_node              0.198        0.20075   3.840  0.409032   
                 occlusion            0.023        0.02300   4.920  0.030280   
                 pagerank             0.599        0.74425   1.490  0.999769   
                 pgmexplainer         0.371        0.41775   3.825  0.982725   
                 random               0.001        0.00100  10.715  0.512361   
                 sa_node              0.109        0.11850   4.690  0.109168   
        0.990    distance             0.724        0.72550   2.395  0.999050   
                 gnnexplainer         0.248        0.24975   6.185  0.971178   
                 ig_node              0.198        0.20075   3.840  0.409032   
                 occlusion            0.023        0.02300   4.920  0.030280   
                 pagerank             0.599        0.74425   1.490  0.999769   
                 pgmexplainer         0.364        0.41025   3.710  0.984609   
                 random               0.002        0.00200  10.580  0.506105   
                 sa_node              0.109        0.11850   4.690  0.109168   
        0.997    distance             0.724        0.72550   2.395  0.999050   
                 gnnexplainer         0.199        0.20025   6.195  0.973600   
                 ig_node              0.198        0.20075   3.840  0.409032   
                 occlusion            0.023        0.02300   4.920  0.030280   
                 pagerank             0.599        0.74425   1.490  0.999769   
                 pgmexplainer         0.374        0.41950   3.755  0.984630   
                 random               0.002        0.00200  10.675  0.510506   
                 sa_node              0.109        0.11850   4.690  0.109168   

                                 f1_score  recall  precision       auc  \
dataset sparsity explainer_name                          

In [34]:
res = res.reset_index()
res.to_csv('/cluster/home/kamara/Explain/csv/syn6_gpu1.csv', index=False)

In [21]:
Expe = ["syn1", "syn3", "syn4", "syn5", "syn6"]

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_sparsity={sparsity}'

df = pd.DataFrame()

for expe in Expe:
    logdir = f'/cluster/home/kamara/Explain/checkpoints/{expe}_gpu1/logs'
    jsonpath = f'/cluster/home/kamara/Explain/config_{expe}.json'
    infos, accuracy_top, accuracy, fidelity = get_info(logdir)
    ranges = get_param_ranges(jsonpath)
    df_infos = get_df_results(infos, ranges, name, metrics = ['time'])
    
    df_acc_top = get_df_results(accuracy_top, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
    df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'auc'])
    df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-'])

    #scores = pd.concat([df_acc, df_fid], axis=1, join="inner")
    print(df_acc_top)
    scores = pd.merge(df_acc_top, df_acc, suffixes=['_top', ''], on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])
    scores = pd.merge(scores, df_fid, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])
    scores = pd.merge(scores, df_infos, on=["dataset", "explainer_name", "num_test_nodes", "data_save_dir", "gpu", "sparsity"])

    print(scores)
    df = pd.concat([df, scores], axis=0)
df

_explainer_name=random_sparsity=0.99
   none explainer_name  sparsity dataset  num_test_nodes data_save_dir   gpu  \
0             distance     0.700    syn1             200          data  True   
1             distance     0.990    syn1             200          data  True   
2             distance     0.997    syn1             200          data  True   
3             pagerank     0.700    syn1             200          data  True   
4             pagerank     0.990    syn1             200          data  True   
5             pagerank     0.997    syn1             200          data  True   
6              sa_node     0.700    syn1             200          data  True   
7              sa_node     0.990    syn1             200          data  True   
8              sa_node     0.997    syn1             200          data  True   
9              ig_node     0.700    syn1             200          data  True   
10             ig_node     0.990    syn1             200          data  True   
11 

KeyError: 'dataset'

In [21]:
df = df.groupby(by=['dataset', 'explainer_name']).mean()
df

num_test_nodes  f1_score    recall  precision    ged  \
dataset explainer_name                                                         
syn1    distance                 100.0  0.706061  0.698333   0.715333   3.76   
        gnnexplainer             100.0  0.343182  0.333333   0.355000   8.19   
        ig_node                  100.0  0.923333  0.923333   0.923333   1.38   
        occlusion                100.0  0.061667  0.061667   0.061667   9.64   
        pagerank                 100.0  0.870000  0.870000   0.870000   2.34   
        pgmexplainer             100.0  0.628333  0.628333   0.628333   5.27   
        random                   100.0  0.003333  0.003333   0.003333  14.34   
        sa_node                  100.0  0.415152  0.415000   0.415333   6.84   
        subgraphx                100.0  0.805848  0.756667   0.865667   3.17   
syn4    distance                 100.0  0.385909  0.378333   0.395000   3.88   
        gnnexplainer             100.0  0.368485  0.361667   0.376667   4.97   
        ig_node                  100.0  0.520455  0.520000   0.521000   3.60   
        occlusion                100.0  0.486667  0.486667   0.486667   5.20   
        pagerank                 100.0  0.520000  0.520000   0.520000   3.72   
        pgmexplainer             100.0  0.596364  0.586667   0.610000   2.51   
        random                   100.0  0.003333  0.003333   0.003333  11.94   
        sa_node                  100.0  0.185121  0.183333   0.187667   7.00   
        subgraphx                100.0  0.382182  0.365000   0.405500   5.87   
syn6    distance                 100.0  0.714000  0.714000   0.714000   2.67   
        gnnexplainer             100.0  0.212444  0.212000   0.213000   6.33   
        ig_node                  100.0  0.282667  0.272000   0.296000   3.75   
        occlusion                100.0  0.002000  0.002000   0.002000   4.26   
        pagerank                 100.0  0.659333  0.594000   0.741000   1.35   
        pgmexplainer             100.0  0.374000  0.356000   0.396500   3.51   
        random                   100.0  0.000000  0.000000   0.000000  10.51   
        sa_node                  100.0  0.273333  0.258000   0.292500   4.07   
        subgraphx                100.0  0.605333  0.592000   0.622000   3.27   

                                 auc  fidelity_acc+  fidelity_acc-  \
dataset explainer_name                                               
syn1    distance        9.983648e-01           0.61           0.00   
        gnnexplainer    9.722994e-01           0.61           0.00   
        ig_node         9.999349e-01           0.61           0.03   
        occlusion       6.023233e-02           0.19           0.43   
        pagerank        9.998975e-01           0.61           0.00   
        pgmexplainer    9.915926e-01           0.61           0.00   
        random          5.011935e-01           0.37           0.58   
        sa_node         5.075639e-01           0.33           0.47   
        subgraphx       9.374829e-01           0.76           0.04   
syn4    distance       -1.000000e+08           0.66           0.00   
        gnnexplainer   -1.000000e+08           0.66           0.00   
        ig_node        -1.000000e+08           0.66           0.01   
        occlusion      -1.000000e+08           0.63           0.55   
        pagerank       -1.000000e+08           0.66           0.00   
        pgmexplainer   -1.000000e+08           0.66           0.00   
        random         -1.000000e+08           0.45           0.64   
        sa_node        -1.000000e+08           0.29           0.64   
        subgraphx      -1.000000e+08           0.66           0.09   
syn6    distance        9.990938e-01           0.62           0.00   
        gnnexplainer    9.659125e-01           0.60           0.00   
        ig_node         3.800498e-01           0.00           0.62   
        occlusion       2.948362e-02           0.01           0.62   
        pagerank       

## Gc Layers

In [50]:
expe = "gc_layers"
logdir = f'/cluster/home/kamara/Explain/checkpoints/{expe}/logs'
jsonpath = f'/cluster/home/kamara/Explain/config_{expe}.json'
infos, accuracy, fidelity = get_info(logdir)
ranges = get_param_ranges(jsonpath)

print(list(accuracy.keys())[1])
name = '{none}_explainer_name={explainer_name}_num_gc_layers={num_gc_layers}'


_explainer_name=pagerank_num_gc_layers=1


In [51]:
df_acc = get_df_results(accuracy, ranges, name, metrics = ['f1_score', 'recall', 'precision', 'ged', 'auc'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-'])
scores = pd.merge(df_acc, df_fid, on=["dataset", "num_gc_layers", "explainer_name", "num_test_nodes", "data_save_dir"])
scores = scores.groupby(by=['num_gc_layers', 'explainer_name']).mean()
scores

num_test_nodes  f1_score    recall  precision  \
num_gc_layers explainer_name                                                  
2             gnnexplainer             200.0  0.682879  0.652500   0.720833   
              pagerank                 200.0  0.864167  0.864167   0.864167   
3             gnnexplainer             200.0  0.329136  0.323333   0.336250   
              pagerank                 200.0  0.864167  0.864167   0.864167   
4             gnnexplainer             200.0  0.055833  0.055833   0.055833   
              pagerank                 200.0  0.860000  0.860000   0.860000   
5             gnnexplainer             200.0  0.006667  0.006667   0.006667   
              pagerank                 200.0  0.859167  0.859167   0.859167   
6             gnnexplainer             200.0  0.003333  0.003333   0.003333   
              pagerank                 200.0  0.859167  0.859167   0.859167   

                                 ged       auc  fidelity_acc+  fidelity_acc-  \
num_gc_layers explainer_name                                                   
2             gnnexplainer     4.065  0.998892          0.835          0.000   
              pagerank         2.435  0.999885          0.835          0.000   
3             gnnexplainer     8.160  0.972214          0.610          0.015   
              pagerank         2.430  0.999885          0.610          0.000   
4             gnnexplainer    12.475  0.834369          0.750          0.290   
              pagerank         2.515  0.999885          0.830          0.000   
5             gnnexplainer    14.085  0.656783          0.550          0.575   
              pagerank         2.525  0.999883          0.640          0.005   
6             gnnexplainer    14.345  0.575590          0.610          0.645   
              pagerank         2.525  0.999883          0.640          0.005   

                              fidelity_prob+  fidelity_prob-  
num_gc_layers explainer_name                                  
2             gnnexplainer          0.650817        0.000000  
              pagerank              0.650817        0.000000  
3             gnnexplainer          0.582833        0.014268  
              pagerank              0.589879       -0.000021  
4             gnnexplainer          0.571764        0.246716  
              pagerank              0.640806        0.005695  
5             gnnexplainer          0.512901        0.548674  
              pagerank              0.608210       -0.001958  
6             gnnexplainer          0.545983        0.581469  
              pagerank              0.579183       -0.000794

## Num top edges

In [10]:
ranges

{'gpu': [True],
 'data_save_dir': ['data'],
 'num_test_nodes': [100],
 'dataset': ['syn1'],
 'num_top_edges': [4, 6, 8, 10, 14, 20],
 'explainer_name': ['distance', 'pagerank', 'gnnexplainer'],
 'none': ['']}

In [7]:
expe = "top_edges"
logdir = f'/cluster/home/kamara/Explain/checkpoints/{expe}/logs'
jsonpath = f'/cluster/home/kamara/Explain/config_{expe}.json'
infos, accuracy, fidelity = get_info(logdir)
ranges = get_param_ranges(jsonpath)
print(list(accuracy.keys())[1])

name = '{none}_explainer_name={explainer_name}_num_top_edges={num_top_edges}'


Unable to replace NaNs
__accuracy:{"sparsity": 0, "normalize": false, "hard_mask": true, "recall": 0.0, "precision": 0.0, "f1_score": 0.0, "ged": 5.19, "auc": 0.5056686188384577}

Unable to replace NaNs
__accuracy:{"sparsity": 0.7, "normalize": false, "hard_mask": true, "recall": 0.0, "precision": 0.0, "f1_score": 0.0, "ged": 5.11, "auc": 0.4954032454856028}

Unable to replace NaNs
__fidelity:{"sparsity": 0, "normalize": false, "hard_mask": true, "fidelity_acc+": 0.61, "fidelity_acc-": 0.0, "fidelity_prob+": 0.5870216488838196, "fidelity_prob-": 0.0}

Unable to replace NaNs
__fidelity:{"sparsity": 0.7, "normalize": false, "hard_mask": true, "fidelity_acc+": 0.22, "fidelity_acc-": 0.44, "fidelity_prob+": 0.22215938568115234, "fidelity_prob-": 0.4264999032020569}

Unable to replace NaNs
__accuracy:{"sparsity": 0, "normalize": false, "hard_mask": true, "recall": 0.0, "precision": 0.0, "f1_score": 0.0, "ged": 13.69, "auc": 0.5056686188384577}

Unable to replace NaNs
__accuracy:{"sparsity":

In [11]:
df_acc = get_df_results(accuracy, ranges, name, metrics = ['sparsity', 'normalize', 'hard_mask', 'f1_score', 'recall', 'precision', 'ged', 'auc'])
df_fid = get_df_results(fidelity, ranges, name, metrics = ['sparsity', 'normalize', 'hard_mask', 'fidelity_acc+', 'fidelity_acc-', 'fidelity_prob+', 'fidelity_prob-'])
scores = pd.merge(df_acc, df_fid, on=["dataset", "num_top_edges", "explainer_name", "num_test_nodes", "data_save_dir", 'sparsity', 'normalize', 'hard_mask'])
scores = scores.groupby(by=['num_top_edges', 'explainer_name']).mean()
scores

num_test_nodes  gpu_x  sparsity  normalize  \
num_top_edges explainer_name                                               
4             distance                 100.0    1.0       0.7        1.0   
              gnnexplainer             100.0    1.0       0.7        1.0   
              pagerank                 100.0    1.0       0.7        1.0   
6             distance                 100.0    1.0       0.7        1.0   
              gnnexplainer             100.0    1.0       0.7        1.0   
              pagerank                 100.0    1.0       0.7        1.0   
8             distance                 100.0    1.0       0.7        1.0   
              gnnexplainer             100.0    1.0       0.7        1.0   
              pagerank                 100.0    1.0       0.7        1.0   
10            distance                 100.0    1.0       0.0        1.0   
              gnnexplainer             100.0    1.0       0.7        1.0   
              pagerank                 100.0    1.0       0.7        1.0   
14            gnnexplainer             100.0    1.0       0.0        1.0   

                              hard_mask  f1_score    recall  precision    ged  \
num_top_edges explainer_name                                                    
4             distance              1.0  0.000000  0.000000   0.000000   5.11   
              gnnexplainer          1.0  0.250444  0.206667   0.319167   4.24   
              pagerank              1.0  0.000000  0.000000   0.000000   5.00   
6             distance              1.0  0.000000  0.000000   0.000000   9.38   
              gnnexplainer          1.0  0.312242  0.305000   0.321333   6.49   
              pagerank              1.0  0.000000  0.000000   0.000000   9.00   
8             distance              1.0  0.000000  0.000000   0.000000  13.54   
              gnnexplainer          1.0  0.341007  0.391667   0.302381   9.37   
              pagerank              1.0  0.000000  0.000000   0.000000  13.00   
10            distance              1.0  0.000000  0.000000   0.000000  17.88   
              gnnexplainer          1.0  0.350405  0.456667   0.284611  11.84   
              pagerank              1.0  0.000000  0.000000   0.000000  17.00   
14            gnnexplainer          1.0  0.335749  0.541667   0.243546  18.44   

                                   auc  gpu_y  fidelity_acc+  fidelity_acc-  \
num_top_edges explainer_name                                                  
4             distance        0.495403    1.0           0.22           0.44   
              gnnexplainer    0.974240    1.0           0.61           0.00   
              pagerank        0.509194    1.0           0.22           0.42   
6             distance        0.495403    1.0           0.22           0.44   
              gnnexplainer    0.974240    1.0           0.61           0.00   
              pagerank        0.509194    1.0           0.22           0.42   
8             distance        0.495403    1.0           0.22           0.44   
              gnnexplainer    0.974240    1.0           0.61           0.00   
              pagerank        0.509194    1.0           0.22           0.42   
10            distance        0.505669    1.0           0.61           0.00   
              gnnexplainer    0.974240    1.0           0.61           0.00   
              pagerank        0.509194    1.0           0.22           0.42   
14            gnnexplainer    0.971840    1.0           0.61           0.00   

                              fidelity_prob+  fidelity_prob-  
num_top_edges explainer_name                                  
4             distance              0.222159        0.426500  
              gnnexplainer          0.587022       -0.000129  
              pagerank              0.223314        0.408714  
6             distance              0.222159        0.426500  
              gnnexplainer          0.587022       -0.000129  
              pagerank              0.223314  

In [12]:
df_acc


,explainer_name,num_top_edges,dataset,num_test_nodes,data_save_dir,gpu,sparsity,normalize,hard_mask,f1_score,recall,precision,ged,auc
0,distance,4,syn1,100,data,True,0.7,True,True,0.000000,0.000000,0.000000,5.11,0.495403
1,distance,6,syn1,100,data,True,0.7,True,True,0.000000,0.000000,0.000000,9.38,0.495403
2,distance,8,syn1,100,data,True,0.7,True,True,0.000000,0.000000,0.000000,13.54,0.495403
3,distance,10,syn1,100,data,True,0.0,True,True,0.000000,0.000000,0.000000,17.88,0.505669
4,pagerank,4,syn1,100,data,True,0.7,True,True,0.000000,0.000000,0.000000,5.00,0.509194
5,pagerank,6,syn1,100,data,True,0.7,True,True,0.000000,0.000000,0.000000,9.00,0.509194
6,pagerank,8,syn1,100,data,True,0.7,True,True,0.000000,0.000000,0.000000,13.00,0.509194
7,pagerank,10,syn1,100,data,True,0.7,True,True,0.000000,0.000000,0.000000,17.00,0.509194
8,gnnexplainer,4,syn1,100,data,True,0.7,True,True,0.250444,0.206667,0.319167,4.24,0.974240
9,gnnexplainer,6,syn1,100,data,True,0.7,True,True,0.312242,0.305000,0.321333,6.49,0.974240
